<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch/blob/lab4-5/Arch_Lab_4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HakureiPOI*** *Arch_Lab_4-5*

---
## *环境准备*

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Thu Dec 26 05:55:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!git clone https://github.com/HakureiPOI/Comp_Arch.git

Cloning into 'Comp_Arch'...
remote: Enumerating objects: 987, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 987 (delta 35), reused 44 (delta 7), pack-reused 900 (from 2)
Receiving objects: 100% (987/987), 62.12 MiB | 17.01 MiB/s, done.
Resolving deltas: 100% (604/604), done.


In [4]:
%cd Comp_Arch

/content/Comp_Arch


In [5]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/lab1
  remotes/origin/lab2
  remotes/origin/lab3
  remotes/origin/lab4-5
  remotes/origin/lab6
  remotes/origin/main


In [6]:
!git switch lab4-5

Branch 'lab4-5' set up to track remote branch 'lab4-5' from 'origin'.
Switched to a new branch 'lab4-5'


In [7]:
%cd lab4-5

/content/Comp_Arch/lab4-5


---
## *Lab4*


### *实现 CUDA 矩阵乘法核函数*

* Step 1: 计算行和列索引
在核函数中，首先需要计算每个线程负责计算的输出矩阵 `P` 的元素位置。这通过 `blockIdx` 和 `threadIdx` 来实现，它们分别表示线程块和线程在网格中的位置。

```cuda
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
```

这里，`blockDim` 定义了每个线程块的尺寸，`blockIdx` 和 `threadIdx` 用于计算每个线程的全局位置。

* Step 2: 确保线程在矩阵边界内
在进行计算之前，需要确保线程计算的位置在矩阵的边界内，以避免越界访问。

```cuda
if (row < width && col < width) {
    float pValue = 0.0;
```

* Step 3: 计算矩阵乘法的单个元素
每个线程计算矩阵 `P` 的一个元素，这是通过遍历矩阵 `M` 的列（或矩阵 `N` 的行）来实现的。

```cuda
for (int k = 0; k < width; ++k) {
    pValue += d_M[row * width + k] * d_N[k * width + col];
}
```

这里，`d_M` 和 `d_N` 分别是输入矩阵 `M` 和 `N` 在设备上的存储，`width` 是矩阵的宽度（对于方阵来说，宽度等于高度）。

* Step 4: 存储计算结果
计算完成后，将结果存储到输出矩阵 `P` 的相应位置。

```cuda
d_P[row * width + col] = pValue;
```


***具体的代码实现如下：***


```cuda
__global__ void MatrixMulKernel(float *d_M, float *d_N, float *d_P, int width) {
  // Calculate the row index of the P element and M
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  // Calculate the column index of the P element and N
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  // Ensure the thread is within bounds
  if (row < width && col < width) {
    float pValue = 0.0;

    // Each thread computes one element of the matrix
    for (int k = 0; k < width; ++k) {
      pValue += d_M[row * width + k] * d_N[k * width + col];
    }

    // Store the computed value into the output matrix
    d_P[row * width + col] = pValue;
  }
}
```



### *尝试编译*

In [8]:
!nvcc -arch=compute_75 -L/usr/local/cuda/lib64 -lcublas ./matrix_mul_lab4.cu

./matrix_mul_lab4.cu(198): warning #177-D: variable "alpha" was declared but never referenced
    const float alpha = 1.0f;
                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

./matrix_mul_lab4.cu(199): warning #177-D: variable "beta" was declared but never referenced
    const float beta = 0.0f;
                ^

./matrix_mul_lab4.cu(18): warning #177-D: variable "BLOCK_SIZE" was declared but never referenced
  const int BLOCK_SIZE = TILE_WIDTH;
            ^



### *脚本测试*

In [9]:
!bash testbench_lab4.sh

Compiling with TILE_SIZE=16
Running tests with TILE_SIZE=16
Matrix size: 128
Kernel Elpased Time: 0.061 ms
Performance= 69.28 GFlop/s, Time= 0.061 msec, Size= 4194304 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Matrix size: 256
Kernel Elpased Time: 0.176 ms
Performance= 190.14 GFlop/s, Time= 0.176 msec, Size= 33554432 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Matrix size: 512
Kernel Elpased Time: 1.149 ms
Performance= 233.61 GFlop/s, Time= 1.149 msec, Size= 268435456 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Matrix size: 1024
Kernel Elpased Time: 9.124 ms
Performance= 235.37 GFlop/s, Time= 9.124 msec, Size= 2147483648 Ops
Computing result using host CPU...done.
Listing first 100 Differences

### *测试结果分析*

***1. TILE_SIZE对性能的影响***

- **TILE_SIZE=16**：
  - 在较小的矩阵大小（128, 256）时，性能相对较低。
  - 随着矩阵大小的增加，性能逐渐提升，但提升幅度不如TILE_SIZE=32和TILE_SIZE=64。

- **TILE_SIZE=32**：
  - 在所有测试的矩阵大小中，性能普遍高于TILE_SIZE=16。
  - 在2048大小的矩阵中，性能达到最高，为306.83 GFlop/s。

- **TILE_SIZE=64**：
  - 性能略低于TILE_SIZE=32，但在某些情况下（如512和1024大小的矩阵）与TILE_SIZE=32相近。
  - 在2048大小的矩阵中，性能为297.75 GFlop/s，略低于TILE_SIZE=32。

***2. 矩阵大小对性能的影响***

- **小矩阵（128, 256）**：
  - 所有TILE_SIZE设置下，小矩阵的性能相对较低。
  - 这可能是因为小矩阵的数据量不足以充分利用并行计算资源。

- **中等矩阵（512, 1024）**：
  - 性能显著提升，尤其是在TILE_SIZE=32和TILE_SIZE=64时。
  - 这表明中等大小的矩阵能更好地利用并行计算资源。

- **大矩阵（2048）**：
  - 性能达到最高，尤其是在TILE_SIZE=32时。
  - 大矩阵能充分利用并行计算资源，尤其是在较大的TILE_SIZE设置下。

***3. 性能趋势和规律***

- **性能提升与矩阵大小的关系**：
  - 随着矩阵大小的增加，性能提升，尤其是在较大的TILE_SIZE设置下。
  - 这表明并行计算核能够有效地扩展以处理更大的数据集。

- **TILE_SIZE的最优值**：
  - TILE_SIZE=32在大多数情况下提供了最佳性能，尤其是在处理大矩阵时。
  - 这可能是因为较大的TILE_SIZE能更有效地利用缓存和并行处理能力。

- **错误检查**：
  - 在所有测试中，总错误数为0，表明核计算的结果与主机CPU的结果一致，验证了核的正确性。

***4. 结论***

- **最优TILE_SIZE选择**：TILE_SIZE=32在大多数情况下提供了最佳性能，尤其是在处理大矩阵时。
- **矩阵大小的影响**：随着矩阵大小的增加，性能提升，表明并行计算核能够有效地扩展以处理更大的数据集。
- **性能一致性**：核的计算结果与主机CPU的结果一致，验证了核的正确性。


---
## *Lab5*

### *实现 CUDA 共享内存矩阵计算优化*

***Step 1: 定义共享内存数组***

首先，我们需要在核函数中定义共享内存数组 `As` 和 `Bs`，它们将用于存储从全局内存中加载的矩阵 A 和 B 的子矩阵。

```cpp
__shared__ float As[TILE_WIDTH][TILE_WIDTH + 1];
__shared__ float Bs[TILE_WIDTH][TILE_WIDTH + 1];
```

***Step 2: 计算线程的全局索引***

接下来，我们需要计算每个线程在矩阵 A 和 B 中的全局索引。这将用于从全局内存中加载数据到共享内存。

```cpp
int aRow = wA * blockIdx.y + threadIdx.y;
int aCol = threadIdx.x;
int bRow = threadIdx.y;
int bCol = wB * blockIdx.x + threadIdx.x;
```

***Step 3: 加载数据到共享内存***

在加载数据之前，我们需要确保线程索引不会超出矩阵的边界。如果超出边界，我们将对应的共享内存元素设置为 0.0f。

```cpp
float a = (aRow < wA && aCol < wA) ? A[aRow * wA + aCol] : 0.0f;
float b = (bRow < wA && bCol < wB) ? B[bRow * wB + bCol] : 0.0f;
As[threadIdx.y][threadIdx.x] = a;
Bs[threadIdx.y][threadIdx.x] = b;
```

***Step 4: 同步线程***

在所有线程完成数据加载后，我们需要同步所有线程，以确保共享内存的数据已经准备好。

```cpp
__syncthreads();
```

***Step 5: 执行矩阵乘法***

现在，我们可以执行矩阵乘法。每个线程将计算其负责的矩阵 C 的元素。

```cpp
float Csub = 0.0f;
for (int k = 0; k < TILE_WIDTH; ++k) {
    Csub += As[threadIdx.y][k] * Bs[k][threadIdx.x];
}
```

***Step 6: 存储结果到全局内存***

最后，我们需要将计算得到的矩阵 C 的元素存储回全局内存。同样，我们需要确保线程索引不会超出矩阵的边界。

```cpp
int cRow = wA * blockIdx.y + threadIdx.y;
int cCol = wB * blockIdx.x + threadIdx.x;
if (cRow < wA && cCol < wB) {
    C[cRow * wB + cCol] = Csub;
}
```


***具体的代码实现如下：***

```cuda
__global__ void MatrixMulSharedMemKernel(float *A, float *B, float *C, int wA, int wB) {
    // 块和线程的索引
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // 初始化Csub变量
    float Csub = 0;

    // 计算A和B的子矩阵
    for (int a = wA * TILE_WIDTH * by, b = TILE_WIDTH * bx; a < wA * TILE_WIDTH * by + wA - 1 && b < TILE_WIDTH * bx + wB - 1; a += TILE_WIDTH, b += TILE_WIDTH) {
        __shared__ float As[TILE_WIDTH][TILE_WIDTH + 1];  // 增加1以减少银行冲突
        __shared__ float Bs[TILE_WIDTH][TILE_WIDTH + 1];  // 增加1以减少银行冲突

        // 加载子矩阵A到共享内存
        int aRow = a / wA + ty;
        int aCol = a % wA + tx;
        if (aRow < wA && aCol < wA)
            As[ty][tx] = A[aRow * wA + aCol];
        else
            As[ty][tx] = 0.0f;

        // 加载子矩阵B到共享内存
        int bRow = b / wB + ty;
        int bCol = b % wB + tx;
        if (bRow < wA && bCol < wB)
            Bs[ty][tx] = B[bRow * wB + bCol];
        else
            Bs[ty][tx] = 0.0f;

        __syncthreads();  // 在加载完子矩阵后同步

        // 进行矩阵乘法
        for (int k = 0; k < TILE_WIDTH; ++k) {
            Csub += As[ty][k] * Bs[k][tx];
        }

        __syncthreads();  // 确保计算完成后再加载下一个子矩阵
    }

    // 存储结果到矩阵C
    int row_C = by * TILE_WIDTH + ty;
    int col_C = bx * TILE_WIDTH + tx;
    if (row_C < wA && col_C < wB)
        C[row_C * wB + col_C] = Csub;
}
```

### *尝试编译*

In [10]:
!nvcc -arch=compute_75 -L/usr/local/cuda/lib64 -lcublas ./matrix_mul_lab5.cu

./matrix_mul_lab5.cu(176): warning #177-D: variable "alpha" was declared but never referenced
      const float alpha = 1.0f;
                  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

./matrix_mul_lab5.cu(177): warning #177-D: variable "beta" was declared but never referenced
      const float beta = 0.0f;
                  ^



### *脚本测试*

In [11]:
!bash testbench_lab5.sh

---****** Testing with cuBLAS ******---

Testing with TILE_WIDTH=16
use cublas with TILE_WIDTH=16

matrix_size: 128, TILE_WIDTH: 16
Kernel Elapsed Time: 0.049 ms
Performance= 84.92 GFlop/s
--------------------------------

matrix_size: 256, TILE_WIDTH: 16
Kernel Elapsed Time: 0.165 ms
Performance= 203.83 GFlop/s
--------------------------------

matrix_size: 512, TILE_WIDTH: 16
Kernel Elapsed Time: 1.011 ms
Performance= 265.53 GFlop/s
--------------------------------

matrix_size: 1024, TILE_WIDTH: 16
Kernel Elapsed Time: 7.813 ms
Performance= 274.86 GFlop/s
--------------------------------

matrix_size: 2048, TILE_WIDTH: 16
Kernel Elapsed Time: 49.061 ms
Performance= 350.17 GFlop/s
--------------------------------

Testing with TILE_WIDTH=32
use cublas with TILE_WIDTH=32

matrix_size: 128, TILE_WIDTH: 32
Kernel Elapsed Time: 0.062 ms
Performance= 68.12 GFlop/s
--------------------------------

matrix_size: 256, TILE_WIDTH: 32
Kernel Elapsed Time: 0.170 ms
Performance= 197.29 GFlop/s
-

### *测试结果分析*

* 1. **TILE_WIDTH的影响：**
   - **TILE_WIDTH** 是优化GPU矩阵运算的重要参数，表示每个块的大小。通过设置适当的 `TILE_WIDTH`，可以平衡内存访问效率与计算资源的利用。
   - 随着 **TILE_WIDTH** 的增大，性能在许多测试中有所提升，特别是当矩阵较大时，性能差异更为明显。

* 2. **不同TILE_WIDTH的性能差异：**
   - **TILE_WIDTH=16：** 对于较小的矩阵（例如 `128x128`），性能相对较高，但随着矩阵尺寸的增加，性能提升趋于平稳。整体来看，使用 `TILE_WIDTH=16` 的性能不是最优的，但适用于较小矩阵。
   - **TILE_WIDTH=32：** 性能相较于 `TILE_WIDTH=16` 略有提升，但并不显著。适用于中等规模矩阵（如 `256x256`、`512x512`）。
   - **TILE_WIDTH=64：** 性能大幅提升，特别是在大矩阵（如 `2048x2048`）时，性能提高显著。这个设置在矩阵较大时表现最好，内核执行时间减少，性能显著提高。

* 3. **使用cuBLAS与不使用cuBLAS的差异：**
   - **cuBLAS**（NVIDIA的BLAS库）显然提升了性能，尤其在较大矩阵时。比如在 `TILE_WIDTH=64` 时，使用 cuBLAS 可以大幅度提高性能，性能数值跳跃式地增加（从几百 GFlop/s 到百万级 GFlop/s）。
   - 不使用 cuBLAS 时，性能会相对较低，尤其是在大矩阵时（如 `2048x2048`）。这表明 cuBLAS 在大规模矩阵乘法中能充分发挥GPU的计算能力和优化优势。

* 4. **性能趋势：**
   - **随着矩阵尺寸增大，性能提高**：在每种 `TILE_WIDTH` 下，随着矩阵大小从 128 增加到 2048，性能普遍呈现上升趋势。更大的矩阵可以更好地利用GPU的并行计算能力，减少了内存访问延迟和处理器空闲时间。
   - **内核执行时间减少**：随着矩阵大小的增大，内核执行时间（`Kernel Elapsed Time`）逐渐减少，表明GPU计算更为高效。

* 总结：
    - 对于大矩阵计算，**TILE_WIDTH=64** 配合 **cuBLAS** 是最佳选择，提供了最高的性能。
    - 不同 `TILE_WIDTH` 的选择主要取决于矩阵的大小，小矩阵选择小的 `TILE_WIDTH` 更为高效，而大矩阵则倾向于选择较大的 `TILE_WIDTH`。
    - 使用 **cuBLAS** 可以显著提高计算性能，尤其在大矩阵时提升更为显著。



---
## *对比分析结果*

通过对比 **使用共享内存优化** 和 **未使用共享内存** 的GPU计算结果，可以观察到几项关键差异。

* 1. **性能对比（GFlop/s）：**
   - **使用共享内存优化的结果**：在矩阵较大时（如 `1024x1024`、`2048x2048`），性能显著提高。例如，在 `TILE_WIDTH=64` 时，性能达到了 **几百万 GFlop/s**，而在 `TILE_WIDTH=16` 时，性能只有 **几十到几百 GFlop/s**。
   - **未使用共享内存的结果**：性能较低，尽管在 `TILE_SIZE=64` 设置下有一些提升，但总体的性能（特别是在大矩阵时）比共享内存优化的测试结果低很多。例如，`TILE_SIZE=64` 时，性能大致在 **200 GFlop/s 到 300 GFlop/s** 之间，远低于共享内存优化后的表现。

* 2. **内核执行时间对比：**
   - **使用共享内存优化的结果**：随着矩阵大小增加，内核执行时间（`Kernel Elapsed Time`）增加，但增幅较小，特别是在 `TILE_WIDTH=64` 时，内核执行时间大大减少。特别是在大矩阵（如 `2048x2048`）时，执行时间仅为 **0.02 ms**，远低于未使用共享内存的时间。
   - **未使用共享内存的结果**：内核执行时间随着矩阵大小增加而显著增加。例如，在 `TILE_SIZE=64` 设置下，`2048x2048` 矩阵的内核执行时间为 **57.7 ms**，大大高于共享内存优化情况下的 **0.02 ms**。

* 3. **TILE_SIZE的影响：**
   - **TILE_SIZE=16**：在未使用共享内存的情况下，`TILE_SIZE=16` 显示出较为平稳的性能，尤其是对于较小的矩阵。对于较小矩阵（如 `128x128`），性能较好。
   - **TILE_SIZE=32 和 TILE_SIZE=64**：随着 `TILE_SIZE` 增加，性能逐渐提升，尤其在较大矩阵时，`TILE_SIZE=64` 性能最高，但依旧比共享内存优化的结果低。

* 4. **优化效果总结：**
   - **共享内存优化**：通过使用共享内存，GPU可以减少对全局内存的访问次数，增加数据的本地性，提高内存访问效率。这对于大矩阵计算尤为重要，能够显著提高性能并减少内核执行时间。
   - **未使用共享内存**：由于计算主要依赖于全局内存访问，导致内存访问效率较低，尤其是对于大矩阵时，性能提升有限，执行时间较长。

* **结论：**
    - 使用共享内存优化GPU代码显著提高了计算性能，尤其在大矩阵情况下，内核执行时间和性能差距非常明显。未使用共享内存时，虽然随着 `TILE_SIZE` 增加性能有所提升，但与共享内存优化的结果相比，差距依然明显。因此，**共享内存优化** 是大规模矩阵运算中不可忽视的重要提升手段。

